## Data collection and imputation

In [61]:
%load_ext autoreload
%autoreload 2

In [46]:
import pandas as pd

In [10]:
df = pd.read_excel('2022-offense.xlsx', sheet_name='Offense_Prior_Actuals', header=3)

# Drop columns that we don't need
drop_columns = ['GS', 'Bye', 'Notes', 'Rank', 'Y! Roto', 'Δ', 'Y! Old', 'Std', 'Δ.1', 'Std Old', 'PPR', 'Δ.2', 'PPR Old', '% Own', 'Fan Pts', 'PPG']
df = df.drop(drop_columns, axis=1)

# Change names of duplicate columns
updated_columns = {'Yds': 'Passing_Yds', 'Yds.1': 'Rushing_Yds', 'Yds.2': 'Receiving_Yds', 'Yds.3': 'Return_Yds', 
                   'TD': 'Passing_Td', 'TD.1': 'Rushing_Td', 'TD.2': 'Receiving_Td', 'TD.3': 'Return_Td', 
                   '1st': 'Passing_1st', '1st.1': 'Rushing_1st', '1st.2': 'Receiving_1st'}
df = df.rename(columns=updated_columns)


Offense_Prior_Actuals
Offense_Proj


/Users/ethancoomber/opt/anaconda3/envs/ml-0451/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize

table_contracts = pd.read_html('https://www.spotrac.com/nfl/contracts/sort-value/offense/all-time/free-agent-2021/limit-100/')

url_pff = 'https://www.pro-football-reference.com/years/2022/passing_advanced.htm'
table_advanced = pd.read_html(url_pff, header=1)[0]

In [98]:
player_contracts = table_contracts[0]
# player_contracts

In [111]:
passing_advanced = table_advanced[0]
passing_advanced = passing_advanced.head(40)
# passing_advanced['Player'] = passing_advanced['Player'].str.replace('*', '')
passing_advanced

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                   Rk             Player                 Tm   
0                   1  Patrick Mahomes*+                KAN   
1                   2     Justin Herbert                LAC   
2                   3          Tom Brady                TAM   
3                   4      Kirk Cousins*                MIN   
4                   5        Joe Burrow*                CIN   
5                   6        Jared Goff*                DET   
6                   7        Josh Allen*                BUF   
7                   8        Geno Smith*                SEA   
8                   9   Trevor Lawrence*                JAX   
9                  10       Jalen Hurts*                PHI   
10                 11      Aaron Rodgers                GNB   
11                 12     Tua Tagovailoa                MIA   
12                 13     Russell Wilson                DEN   
13                 14        Derek Carr*                LVR   
14                 15       Daniel Jones                NYG   
15                 16        Davis Mills                HOU   
16                 17          Matt Ryan                IND   
17                 18          Mac Jones                NWE   
18                 19        Andy Dalton                NOR   
19                 20       Dak Prescott                DAL   
20                 21    Jacoby Brissett                CLE   
21                 22     Ryan Tannehill                TEN   
22                 23    Jimmy Garoppolo                SFO   
23                 24      Kenny Pickett                PIT   
24                 25       Kyler Murray                ARI   
25                 26      Justin Fields                CHI   
26                 27      Lamar Jackson                BAL   
27                 28     Marcus Mariota                ATL   
28                 29     Baker Mayfield                2TM   
29                 30   Matthew Stafford                LAR   
30                 31    Taylor Heinicke                WAS   
31                 32       Carson Wentz                WAS   
32                 33        Zach Wilson                NYJ   
33                 34        Brock Purdy                SFO   
34                 35  Mitchell Trubisky                PIT   
35                 36         Mike White                NYJ   
36                 37        Sam Darnold                CAR   
37                 38     Deshaun Watson                CLE   
38                 39         Joe Flacco                NYJ   
39                 40        Cooper Rush                DAL   

   Unnamed: 3_level_0 Unnamed: 4_level_0 Games     Passing                   \
                  Age                Pos     G  GS     Cmp  Att   Yds   IAY   
0                  27                 QB    17  17     435  648  5250  4686   
1                  24                 QB    17  17     477  699  4739  4476   
2                  45                 QB    17  17     490  733  4694  4876   
3                  34                 QB    17  17     424  643  4547  4800   
4                  26                 QB    16  16     414  606  4475  4121   
5                  28                 QB    17  17     382  587  4438  4106   
6                  26                 QB    16  16     359  567  4283  5205   
7                  32                 QB    17  17     399  572  4282  4325   
8                  23                 QB    17  17     387  584  4113  4328   
9                  24                 QB    15  15     306  460  3701  3714   
10                 39                 QB    17  17     350  542  3695  4341   
11                 24                 QB    13  13     259  400  3548  3790   
12                 34                 QB    15  15     292  483  3524  4270   
13                 31                 QB    15  15     305  502  3522  4589   
14                 25                 QB    16  16     317  472  3205  3024   
15           

In [93]:
# Get y - actual values from the next year (we do not have 2023 but could get them) - we will eventually need them?
QB_data = df[df["Pos"] == "QB"]

# Get the list of all column names from headers
qb_column_headers = list(QB_data.columns.values)
# print("The Column Header :", qb_column_headers)

qb_drop_columns = ['Tgt', 'Rec', 'Receiving_Yds', 'Receiving_Td', 'Receiving_1st', 'Return_Yds', 'Return_Td', '2PT',]
QB_data = QB_data.drop(qb_drop_columns, axis=1)
# QB_data



In [96]:
from fantasy_football import Fantasy

football = Fantasy()
football.addContracts(QB_data, player_contracts)

In [24]:
RB_data = df[df["Pos"] == "RB"]

rb_drop_columns = [ 'Comp', 'Inc', 'Passing_Yds', 'Passing_Td', 'Int', 'Pic6', 'Sks', 'Passing_1st']
RB_data = RB_data.drop(rb_drop_columns, axis=1)
# RB_data

In [23]:
WR_data = df[df["Pos"] == "WR"]

wr_drop_columns = ['Comp', 'Inc', 'Passing_Yds', 'Passing_Td', 'Int', 'Pic6', 'Sks', 'Passing_1st']
WR_data = WR_data.drop(rb_drop_columns, axis=1)
# WR_data

In [25]:
TE_data = df[df["Pos"] == "TE"]

te_drop_columns = ['Comp', 'Inc', 'Passing_Yds', 'Passing_Td', 'Int', 'Pic6', 'Sks', 'Passing_1st', 'Return_Yds', 'Return_Td']
TE_data = TE_data.drop(te_drop_columns, axis=1)
TE_data

,Player,Pos,Team,GP,Att,Rushing_Yds,Rushing_Td,Rushing_1st,Tgt,Rec,Receiving_Yds,Receiving_Td,Receiving_1st,2PT,Total,Lost
45,M. Andrews,TE,Bal,17,1,0,0,0,153,107,1361,9,75,2,1,0
62,T. Kelce,TE,KC,16,2,3,1,1,134,92,1125,9,63,0,1,1
96,D. Schultz,TE,Dal,17,0,0,0,0,104,78,808,8,43,1,0,0
98,G. Kittle,TE,SF,14,3,20,0,0,94,71,910,6,42,0,2,1
111,K. Pitts,TE,Atl,17,0,0,0,0,110,68,1026,1,43,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,J. Akins,TE,NYG,13,1,3,0,1,33,24,214,0,11,0,2,2
356,Z. Gentry,TE,Pit,17,0,0,0,0,25,19,167,0,11,0,0,0
358,E. Ebron,TE,Pit,8,1,1,1,1,18,12,84,1,6,0,0,0
365,N. Vannett,TE,NO,7,0,0,0,0,15,9,133,1,7,0,0,0
